In [92]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys

# third party imports 
import numpy as np 

import pandas as pd
import random
import nibabel as nb
import cv2
import ast
from scipy.linalg import sqrtm
from scipy.ndimage import center_of_mass
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

import torch
import torchvision.models as models
import torch.nn.functional as F

import Diffusion_motion_field.Build_lists.Build_list as Build_list
import Diffusion_motion_field.functions_collection as ff
from Diffusion_motion_field.common_metrics_on_video_quality.fvd.styleganv.fvd import get_fvd_feats, frechet_distance, load_i3d_pretrained

main_path = '/mnt/camca_NAS/4DCT/models'

### get patient list

In [74]:
# load patient list
data_sheet = os.path.join(os.path.dirname(main_path),'Patient_lists/mgh/patient_list_MVF_diffusion_train_test.xlsx')
b = Build_list.Build(data_sheet)
patient_class_list, patient_id_list,_ = b.__build__(batch_list = [5])
# patient_class_list = patient_class_list[0:2]
# patient_id_list = patient_id_list[0:2]

timeframe_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx')

trial_name = 'MVF_EDM_down_10tf_imgcon_EFcon'
epoch = 2350


### Prepare all data for original and generated

In [47]:
original_data = np.zeros((len(patient_id_list), 10,128,128,96))
generated_data_factual = np.zeros((len(patient_id_list), 10,128,128,96))
generated_data_counterfactual = np.zeros((len(patient_id_list),3,10, 128,128,96))

In [32]:
# for i in range(len(patient_id_list)):

#     patient_class = patient_class_list[i]
#     patient_id = patient_id_list[i]

#     row = timeframe_info[timeframe_info['patient_id'] == patient_id]
#     sampled_time_frame_list = ast.literal_eval(row['sampled_time_frame_list'].iloc[0])
#     normalized_time_frame_list = ast.literal_eval(row['normalized_time_frame_list_copy'].iloc[0])
#     picked_tf_normalized = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#     picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]  

#     # get the segmentation
#     seg_file = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class, patient_id, 'pred_s_0.nii.gz')
#     seg = nb.load(seg_file).get_fdata(); seg = np.round(seg); seg[seg!= 1] = 0
#     # find the center of LV = 1
#     center = center_of_mass(seg)
#     center_x, center_y, center_z = int(center[0]), int(center[1]), int(center[2])
#     x_start = max(0, center_x - 64); x_end = x_start + 128
#     y_start = max(0, center_y - 64); y_end = y_start + 128


#     # get original data
#     folder = os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class, patient_id, 'image_warped_mvf_original')
#     for tf_n in range(0, len(picked_tf)):
#         original_data[i, tf_n, ... ] = nb.load(os.path.join(folder, str(picked_tf[tf_n]) + '.nii.gz')).get_fdata()[x_start:x_end, y_start:y_end, :]
    
#     # get generated data factural
#     folder = os.path.join('/mnt/camca_NAS/4DCT/models/',trial_name,'pred_mvf', patient_class, patient_id,  'epoch' + str(epoch)+ '_0')
#     for tf_n in range(0, len(picked_tf)):
#         generated_data_factual[i, tf_n, ...] = nb.load(os.path.join(folder, 'warped_img_pred_tf' + str(picked_tf[tf_n]) + '.nii.gz')).get_fdata()[x_start:x_end, y_start:y_end, :]
    
#     # get generated data counterfactual
#     folder = os.path.join('/mnt/camca_NAS/4DCT/models/',trial_name,'pred_mvf', patient_class, patient_id,  'epoch' + str(epoch)+  '_1')
#     for tf_n in range(0,len(picked_tf)):
#         generated_data_counterfactual[i, 0,tf_n, ...] = nb.load(os.path.join(folder, 'warped_img_pred_tf' + str(picked_tf[tf_n]) +'.nii.gz')).get_fdata()[x_start:x_end, y_start:y_end, :]
#     folder = os.path.join('/mnt/camca_NAS/4DCT/models/',trial_name,'pred_mvf', patient_class, patient_id,  'epoch' + str(epoch)+  '_2')
#     for tf_n in range(0,len(picked_tf)):
#         generated_data_counterfactual[i, 1,tf_n, ...] = nb.load(os.path.join(folder, 'warped_img_pred_tf' + str(picked_tf[tf_n]) +'.nii.gz')).get_fdata()[x_start:x_end, y_start:y_end, :]
#     folder = os.path.join('/mnt/camca_NAS/4DCT/models/',trial_name,'pred_mvf', patient_class, patient_id,  'epoch' + str(epoch)+  '_3')
#     for tf_n in range(0,len(picked_tf)):
#         generated_data_counterfactual[i, 2,tf_n, ...] = nb.load(os.path.join(folder, 'warped_img_pred_tf' + str(picked_tf[tf_n]) +'.nii.gz')).get_fdata()[x_start:x_end, y_start:y_end, :]
   

In [70]:
if os.path.isfile(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'original_data_sampled_for_quantitative.npy')) == 0:
  
    original_data_sampled = original_data[...,::3]
    generated_data_factual_sampled = generated_data_factual[...,::3]
    generated_data_counterfactual_sampled = generated_data_counterfactual[...,::3]
    print('original_data_sample shape:', original_data_sampled.shape)

    np.save(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'original_data_sampled_for_quantitative.npy'), original_data_sampled)
    np.save(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'generated_data_factual_sampled_for_quantitative.npy'), generated_data_factual_sampled)
    np.save(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'generated_data_counterfactual_sampled_for_quantitative.npy'), generated_data_counterfactual_sampled)

else:
    print('load existing data')
    original_data_sampled = np.load(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'original_data_sampled_for_quantitative.npy'))
    generated_data_factual_sampled = np.load(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'generated_data_factual_sampled_for_quantitative.npy'))
    generated_data_counterfactual_sampled = np.load(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'generated_data_counterfactual_sampled_for_quantitative.npy'))


load existing data


### FID

In [17]:
# Load pretrained InceptionV3
inception = models.inception_v3(pretrained=True)
inception.fc = torch.nn.Identity()  # Remove classification head
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception.eval().to(device)

def extract_inception_features(slices_np):  # [N, 1, H, W]
    tensor = torch.from_numpy(slices_np).float()
    tensor = tensor / 255.0  # Normalize
    tensor = F.interpolate(tensor, size=(299, 299), mode='bilinear')  # Resize
    tensor = tensor.repeat(1, 3, 1, 1)  # Convert to RGB
    with torch.no_grad():
        feats = inception(tensor.cuda())  # [N, 2048]
    return feats.cpu().numpy()

def calculate_fid(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean = sqrtm(sigma1 @ sigma2)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff @ diff + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
original_features_list = []
factual_features_list = []
counterfactual_features_list = []
for i in range(0, original_data_sampled.shape[0]):
    
    # Extract features for original data
    original_patient = original_data_sampled[i, :, :, :, :].reshape(-1, 1, 128, 128)  # Reshape to [N, 1, H, W]
    original_features_list.append(extract_inception_features(original_patient))

    # Extract features for generated factual data
    factual_patient = generated_data_factual_sampled[i, :, :, :, :].reshape(-1, 1, 128, 128)  # Reshape to [N, 1, H, W]
    factual_features_list.append(extract_inception_features(factual_patient))

    # Extract features for generated counterfactual data
    counterfactual_patient = generated_data_counterfactual_sampled[i, :, :, :, :].reshape(-1, 1, 128, 128)  # Reshape to [N, 1, H, W]
    counterfactual_features_list.append(extract_inception_features(counterfactual_patient))

# Convert lists to numpy arrays
original_features = np.array(original_features_list).reshape(-1, 2048)
factual_features = np.array(factual_features_list).reshape(-1, 2048)
counterfactual_features = np.array(counterfactual_features_list).reshape(-1, 2048)

    

In [19]:
mu_original = np.mean(original_features, axis=0)
sigma_original = np.cov(original_features, rowvar=False)
mu_factual = np.mean(factual_features, axis=0)
sigma_factual = np.cov(factual_features, rowvar=False)
mu_counterfactual = np.mean(counterfactual_features, axis=0)
sigma_counterfactual = np.cov(counterfactual_features, rowvar=False)

fid_factual = calculate_fid(mu_original, sigma_original, mu_factual, sigma_factual)
fid_counterfactual = calculate_fid(mu_original, sigma_original, mu_counterfactual, sigma_counterfactual)
print('FID factual:', fid_factual)
print('FID counterfactual:', fid_counterfactual)

FID factual: 11.699855959981884
FID counterfactual: 10.53149512930959


### FVD

In [49]:
def trans(x):
    # if greyscale images add channel
    if x.shape[-3] == 1:
        x = x.repeat(1, 1, 3, 1, 1)

    # permute BTCHW -> BCTHW
    x = x.permute(0, 2, 1, 3, 4) 

    return x

def calculate_fvd(videos1, videos2, device, only_final=False):

    # videos [batch_size, timestamps, channel, h, w]
    
    assert videos1.shape == videos2.shape

    i3d = load_i3d_pretrained(device=device)
    fvd_results = []

    # support grayscale input, if grayscale -> channel*3
    # BTCHW -> BCTHW
    # videos -> [batch_size, channel, timestamps, h, w]

    videos1 = trans(videos1)
    videos2 = trans(videos2)

    # print('after trans videos1 shape: ', videos1.shape, ' videos2 shape: ', videos2.shape)

    fvd_results = []

    assert videos1.shape[2] >= 10, "for calculate FVD, each clip_timestamp must >= 10"

    # videos_clip [batch_size, channel, timestamps, h, w]
    videos_clip1 = videos1
    videos_clip2 = videos2

    # get FVD features
    feats1 = get_fvd_feats(videos_clip1, i3d=i3d, device=device)
    feats2 = get_fvd_feats(videos_clip2, i3d=i3d, device=device)

    # calculate FVD
    fvd_results.append(frechet_distance(feats1, feats2))
    
    result = {
        "value": fvd_results,
    }

    return result

In [35]:
# random_arr = np.random.randint(0, 3, size=40)
# np.save(os.path.join('/mnt/camca_NAS/4DCT/models/', 'MVF_EDM_down_10tf_imgcon_EFcon_warp_orires', 'random_pick_counterfactual_for_FVD.npy'), random_arr)

In [54]:
# do FVD slice by slice
random_arr = np.load(os.path.join('/mnt/camca_NAS/4DCT/models/', 'MVF_EDM_down_10tf_imgcon_EFcon_warp_orires', 'random_pick_counterfactual_for_FVD.npy'))

result_fvd_factual = []
result_fvd_counterfactual = []
for slice_n in range(0,original_data_sampled.shape[-1]):
    original_data_sampled_normalized = (original_data_sampled - np.min(original_data_sampled)) / (np.max(original_data_sampled) - np.min(original_data_sampled))
    generated_data_factual_sampled_normalized = (generated_data_factual_sampled - np.min(generated_data_factual_sampled)) / (np.max(generated_data_factual_sampled) - np.min(generated_data_factual_sampled))
    generated_data_counterfactual_sampled_normalized_raw = (generated_data_counterfactual_sampled - np.min(generated_data_counterfactual_sampled)) / (np.max(generated_data_counterfactual_sampled) - np.min(generated_data_counterfactual_sampled))
    
    # for each counterfactual, pick one sample (since this fvd function needs same shape for video 1 and video 2)
    generated_data_counterfactual_sampled_normalized = np.zeros_like(generated_data_counterfactual_sampled_normalized_raw[:,0,...])
    for ii in range(0, generated_data_counterfactual_sampled_normalized.shape[0]):
        generated_data_counterfactual_sampled_normalized[ii, ...] = generated_data_counterfactual_sampled_normalized_raw[ii, random_arr[ii], ...]

    original_data_slice = torch.from_numpy(original_data_sampled_normalized[..., slice_n]).unsqueeze(2).float()
    generated_data_factual_slice = torch.from_numpy(generated_data_factual_sampled_normalized[..., slice_n]).unsqueeze(2).float()
    generated_data_counterfactual_slice = torch.from_numpy(generated_data_counterfactual_sampled_normalized[..., slice_n]).unsqueeze(2).float()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    result_factual = calculate_fvd(original_data_slice, generated_data_factual_slice, device)
    result_counterfactual = calculate_fvd(original_data_slice, generated_data_counterfactual_slice, device)
    result_fvd_factual.append(result_factual['value'][0])
    result_fvd_counterfactual.append(result_counterfactual['value'][0])

    print('slice: ', slice_n, ' FVD factual: ', result_fvd_factual[-1], ' FVD counterfactual: ', result_fvd_counterfactual[-1])

df = pd.DataFrame({'slice_sampled': list(range(0,original_data_sampled.shape[-1])),
                   'fvd_factual': result_fvd_factual, 
                   'fvd_counterfactual': result_fvd_counterfactual})
df.to_excel(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'fvd_results.xlsx'), index=False)

### SSIM, MAE, RMSE

In [71]:
results = []
for i in range(0, original_data_sampled.shape[0]):
    patient_class = patient_class_list[i]
    patient_id = patient_id_list[i]
    ssim_factual_patient = []
    mae_factual_patient = []
    rmse_factual_patient = []
    for tf in range(0, original_data_sampled.shape[1]):
        mae, _, rmse, _, ssim = ff.compare(original_data_sampled[i, tf, ...], generated_data_factual_sampled[i, tf, ...])
        ssim_factual_patient.append(ssim)
        mae_factual_patient.append(mae)
        rmse_factual_patient.append(rmse)
    ssim_factual_patient = sum(ssim_factual_patient) / len(ssim_factual_patient)
    mae_factual_patient = sum(mae_factual_patient) / len(mae_factual_patient)
    rmse_factual_patient = sum(rmse_factual_patient) / len(rmse_factual_patient)

    results.append([patient_class, patient_id, ssim_factual_patient, mae_factual_patient, rmse_factual_patient])
    df_results = pd.DataFrame(results, columns=['patient_class', 'patient_id', 'ssim', 'mae', 'rmse'])
    df_results.to_excel(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'ssim_mae_rmse_factual.xlsx'), index=False)

In [72]:
results = []
for i in range(0, original_data_sampled.shape[0]):
    for round_test in range(0, 3):
        patient_class = patient_class_list[i]
        patient_id = patient_id_list[i]
        ssim_factual_patient = []
        mae_factual_patient = []
        rmse_factual_patient = []
        for tf in range(0, original_data_sampled.shape[1]):
            mae, _, rmse, _, ssim = ff.compare(original_data_sampled[i, tf, ...], generated_data_counterfactual_sampled[i, round_test,tf, ...])
            ssim_factual_patient.append(ssim)
            mae_factual_patient.append(mae)
            rmse_factual_patient.append(rmse)
        ssim_factual_patient = sum(ssim_factual_patient) / len(ssim_factual_patient)
        mae_factual_patient = sum(mae_factual_patient) / len(mae_factual_patient)
        rmse_factual_patient = sum(rmse_factual_patient) / len(rmse_factual_patient)

        results.append([patient_class, patient_id,round_test, ssim_factual_patient, mae_factual_patient, rmse_factual_patient])
        df_results = pd.DataFrame(results, columns=['patient_class', 'patient_id', 'round_test','ssim', 'mae', 'rmse'])
        df_results.to_excel(os.path.join('/mnt/camca_NAS/4DCT/models/', trial_name, 'ssim_mae_rmse_counterfactual.xlsx'), index=False)

### EF

In [98]:
# ef_list = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/models/', 'MVF_EDM_down_10tf_imgcon_EFcon', 'EF_from_warploss_results_epoch2350_edited.xlsx'))
# ef_list = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/models/', 'MVF_EDM_down_10tf_imgcon_EFcon_EFpredict_trial2', 'EF_from_warploss_results_epoch1410_edited.xlsx'))
ef_list = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/models/', 'MVF_EDM_down_10tf_imgcon_EFcon_warp_orires', 'EF_from_warploss_results_epoch1930_edited.xlsx'))
ef_list = ef_list[:-1]
ef_list = ef_list[['patient_class', 'patient_id', 'round_test', 'preset_EF', 'pred_EF']]

# get a new column for abs difference
ef_list['diff'] = abs(ef_list['preset_EF'] - ef_list['pred_EF'])

ef_list_factual = ef_list[ef_list['round_test'] == 0]
ef_list_counterfactual = ef_list[ef_list['round_test'] != 0]
print('ef_list_factual shape: ', ef_list_factual.shape, 'ef_list_counterfactual shape: ', ef_list_counterfactual.shape)

ef_list_factual shape:  (40, 6) ef_list_counterfactual shape:  (120, 6)


In [99]:
# calculate the mean and std 
factual_mean = ef_list_factual['diff'].mean()
factual_std = ef_list_factual['diff'].std()
r2 = r2_score(ef_list_factual["preset_EF"], ef_list_factual["pred_EF"])
r, p_value = pearsonr(ef_list_factual["preset_EF"], ef_list_factual["pred_EF"])
print('Factual EF abs diff mean: ', factual_mean, ' std: ', factual_std, ' correlation: ', r2, ' pearson r: ', r, ' p-value: ', p_value)

Factual EF abs diff mean:  0.024431153483688833  std:  0.02080821568749653  correlation:  0.9612665416914933  pearson r:  0.9897574796641788  p-value:  9.735754937768578e-34


In [100]:
# calculate the mean and std for counterfactual
counterfactual_mean = ef_list_counterfactual['diff'].mean()
counterfactual_std = ef_list_counterfactual['diff'].std()
r2_counterfactual = r2_score(ef_list_counterfactual["preset_EF"], ef_list_counterfactual["pred_EF"])
r_counterfactual, p_value_counterfactual = pearsonr(ef_list_counterfactual["preset_EF"], ef_list_counterfactual["pred_EF"])
print('Counterfactual EF abs diff mean: ', counterfactual_mean, ' std: ', counterfactual_std, ' correlation: ', r2_counterfactual, ' pearson r: ', r_counterfactual, ' p-value: ', p_value_counterfactual)

Counterfactual EF abs diff mean:  0.025082963934168218  std:  0.01938759338660615  correlation:  0.9738940654257792  pearson r:  0.9952109461575658  p-value:  5.025770755091874e-121
